In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('combined-dataset/final_reviews_data.csv')

# Encode user_id and place_id
user_encoder = LabelEncoder()
place_encoder = LabelEncoder()

df['user_id_encoded'] = user_encoder.fit_transform(df['user_id'])
df['place_id_encoded'] = place_encoder.fit_transform(df['id'])

# Display the DataFrame with encoded columns
df.head()


,id,types,review_number,review,user_id,sentiment,user_id_encoded,place_id_encoded
0,ChIJYcGr7GSb0S0RckePBrCWikw,"hotel, lodging",review 1,"It has quite small room, and the hallway is qu...",user_18425,3.02,3703,3660
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"indonesian_restaurant, restaurant, food",review 1,"Surprisingly, a really good warung that’s hidd...",user_46108,3.42,15699,3762
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"tourist_attraction, restaurant, food",review 1,"Only had a fleeting visit here, came by coach,...",user_43063,2.97,14376,3678
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"coffee_shop, cafe, food, store",review 1,"One word, underrated! How come place like this...",user_18528,3.69,3748,829
4,ChIJxaITmQia0S0RyrbukE8vsJU,"tourist_attraction, place_of_worship",review 1,"This temple is located in Singaraja, located i...",user_19259,3.28,4073,6310


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Dense, Input, Concatenate
from tensorflow.keras.models import Model

# Number of unique users and places
num_users = df['user_id_encoded'].nunique()
num_places = df['place_id_encoded'].nunique()

# Define the embedding size
embedding_size = 16

# User input
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(num_users, embedding_size, name='user_embedding')(user_input)
user_vec = Flatten(name='user_flatten')(user_embedding)

# Place input
place_input = Input(shape=(1,), name='place_input')
place_embedding = Embedding(num_places, embedding_size, name='place_embedding')(place_input)
place_vec = Flatten(name='place_flatten')(place_embedding)

# Concatenate user and place vectors
concat = Concatenate()([user_vec, place_vec])

# Add dense layers
dense = Dense(128, activation='relu')(concat)
output = Dense(1, activation='linear')(dense)

# Build the model
model = Model(inputs=[user_input, place_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display the model summary
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 place_input (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 16)                411072    ['user_input[0][0]']          
                                                                                                  
 place_embedding (Embedding  (None, 1, 16)                104768    ['place_input[0][0]']         
 )                                                                                           

In [3]:
# Train the model
history = model.fit(
    [df['user_id_encoded'], df['place_id_encoded']],
    df['sentiment'],
    epochs=10,
    batch_size=1,
    verbose=1
)


Epoch 1/10
32745/32745 [==============================] - 164s 5ms/step - loss: 0.2678
Epoch 2/10
32745/32745 [==============================] - 115s 4ms/step - loss: 0.2012
Epoch 3/10
32745/32745 [==============================] - 120s 4ms/step - loss: 0.1448
Epoch 4/10
32745/32745 [==============================] - 126s 4ms/step - loss: 0.1123
Epoch 5/10
32745/32745 [==============================] - 124s 4ms/step - loss: 0.0899
Epoch 6/10
32745/32745 [==============================] - 117s 4ms/step - loss: 0.0726
Epoch 7/10
32745/32745 [==============================] - 110s 3ms/step - loss: 0.0576
Epoch 8/10
32745/32745 [==============================] - 174s 5ms/step - loss: 0.0474
Epoch 9/10
32745/32745 [==============================] - 104s 3ms/step - loss: 0.0392
Epoch 10/10
32745/32745 [==============================] - 107s 3ms/step - loss: 0.0331


In [4]:
# Evaluate the model
loss = model.evaluate([df['user_id_encoded'], df['place_id_encoded']], df['sentiment'])
print(f"Model Loss: {loss}")


1024/1024 [==============================] - 1s 1ms/step - loss: 0.0288
Model Loss: 0.028812170028686523


In [5]:
model.save('39_collab_model.keras')

In [6]:
import numpy as np

# Function to recommend places for a given user
def recommend_places(user_id, model, df, user_encoder, place_encoder, top_n=5):
    user_idx = user_encoder.transform([user_id])[0]
    all_place_ids = df['place_id_encoded'].unique()
    user_array = np.array([user_idx] * len(all_place_ids))
    
    predictions = model.predict([user_array, all_place_ids])
    
    place_indices = predictions.flatten().argsort()[-top_n:][::-1]
    recommended_places = place_encoder.inverse_transform(place_indices)
    
    return recommended_places

# Get recommendations for a specific user
user_id = "user_18425"
recommended_places = recommend_places(user_id, model, df, user_encoder, place_encoder)
print(f"Recommended places for user {user_id}: {recommended_places}")

places = pd.read_csv('combined-dataset/combined_datasetV2.csv')
places = places[places['id'].isin(recommended_places)]  # Filter places
places


205/205 [==============================] - 0s 1ms/step
Recommended places for user user_18425: ['ChIJM_7mcTZz0i0RDxK41mExGTk' 'ChIJqZ2jWOMX0i0Rr6xvzG8HXjI'
 'ChIJtyiHqSRD0i0RSJxxpT8u4FM' 'ChIJkUCRuTEm0i0R5EocJ4O1NdA'
 'ChIJO2aDqyVH0i0R5ac30dOPAUc']


,id,name,latitude,longitude,address,url,status,phone,primary-type,types,rating,rating-count,price-level,review 1,review 2,review 3,review 4,review 5
9357,ChIJkUCRuTEm0i0R5EocJ4O1NdA,TEGAL JERO Homestay,-8.383688,115.108480,"Jalan Penatahan - Wongaya Gede, J485+G9H, Pene...",https://maps.google.com/?cid=15003097308556184292,OPERATIONAL,+62 852-3809-0000,lodging,lodging,4.5,88.0,NaN,My first trip to bali.i flew from Australia an...,🌿✨ Tranquil Retreat in Nature: TEGAL JERO Home...,"This is a good place to stay , i suggest you t...",Adi is an amazing host with a simple but cute ...,We can definitely recommend this place! We had...
21003,ChIJqZ2jWOMX0i0Rr6xvzG8HXjI,Tomato Wood Fired Pizza And Pasta Gianyar,-8.541505,115.317586,"Jl. Kenarok No.6, Gianyar, Kec. Gianyar, Kabup...",https://maps.google.com/?cid=3629346526459833519,OPERATIONAL,+62 821-4581-8567,restaurant,"restaurant, food",4.3,97.0,NaN,"Discovered through an Instagram ad, this resta...",+ the best pizza I've had in Bali\n+ the best ...,"The pasta is very tasty, very cheap, the porti...","We went specifically to get the ""Pragiatha"" pi...",It's decent if you're craving for a quick pizz...
29383,ChIJM_7mcTZz0i0RDxK41mExGTk,My Art Hostel,-8.676856,115.552880,"Unnamed Road, Kutampi Kaler, Nusa Penida, Klun...",https://maps.google.com/?cid=4114374030863634959,OPERATIONAL,+62 813-3908-2551,hostel,"hostel, lodging",4.3,92.0,NaN,Amazing tasty breakfast! Nice big beds and in ...,Amazing breakfast! The staff super nice and fr...,My art hostel was a super nice place with a gr...,"Great hostel.\nGood rooms with big beds, clean...","Comfy spacious rooms, cute small pool. The sta..."
29461,ChIJO2aDqyVH0i0R5ac30dOPAUc,KamarKoe Hotel Seminyak,-8.686134,115.174480,"Jl. Dewi Saraswati III No.53-55, Seminyak, Kec...",https://maps.google.com/?cid=5116528791580551141,OPERATIONAL,+62 361 4741953,hotel,"hotel, lodging",4.5,155.0,NaN,Good modest hotel. Spacious room. Good service...,"Based on my experience, the hotel atmosphere w...",The best room we have stayed so far. Super cle...,I stayed in this hotel for one month or even m...,Great staff always willing to help! Beds are r...
31897,ChIJtyiHqSRD0i0RSJxxpT8u4FM,Amaris Hotel Pratama Nusa Dua - Bali,-8.796254,115.224730,"Jl. Pratama, Banjar Celuk, Benoa, Kec. Kuta Se...",https://maps.google.com/?cid=6043881550824709192,OPERATIONAL,+62 361 8468333,hotel,"hotel, lodging",4.1,1215.0,NaN,"Stayed with family for a few nights, great pri...",For me the price is so unbeatable compared to ...,A decent and cost effective solution in the Nu...,Friendly staffs & thanks to Mba Oliep for taki...,Perfect for the price. Very kind and friendly ...
